In [ ]:
#@title Environment Setup
import torch
if torch.cuda.is_available() == True:
  "CUDA device name:", torch.cuda.get_device_name(0)
  !nvidia-smi
else:
  import sys
  print("No CUDA GPU found. Did you forget to use the correct runtime type?")
  sys.exit()

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
INSTALL_COMFYUI_MANAGER = False  #@param {type:"boolean"}
INSTALL_ANIMATEDIFF = False  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = False  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['INSTALL_COMFYUI_MANAGER'] = INSTALL_COMFYUI_MANAGER
OPTIONS['INSTALL_ANIMATEDIFF'] = INSTALL_ANIMATEDIFF
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

if OPTIONS['USE_GOOGLE_DRIVE']:
  print("Mounting Google Drive...")
  %cd /

  from google.colab import drive
  drive.mount('/content/drive')

  AI_WORKSPACE = "/content/drive/MyDrive/temp"
  COMFY_WORKSPACE = "/content/drive/MyDrive/temp/ComfyUI"
else:
  current_dir = !pwd
  AI_WORKSPACE = f"{current_dir[0]}/temp"
  COMFY_WORKSPACE = f"{current_dir[0]}/temp/ComfyUI"

import os
print("Creating AI workspace (if it doesn't exist)")
os.makedirs(AI_WORKSPACE, exist_ok=True)

%cd $AI_WORKSPACE

![ ! -d $COMFY_WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $COMFY_WORKSPACE
   
!nvidia-smi

if OPTIONS['UPDATE_COMFY_UI']:
  print("-= Updating ComfyUI =-")
  !git pull

print("-= Install dependencies =-")
# that download link might be getting out of date version
%pip install https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117
%pip install onnxruntime-gpu color-matcher simpleeval
%pip install --use-pep517 facexlib
%pip install onnxruntime-gpu==1.15.1
%pip install insightface onnxruntime

import onnxruntime as ort

available_providers = ort.get_available_providers()
print("Available providers:", available_providers)

if "CUDAExecutionProvider" in available_providers:
    print("CUDAExecutionProvider is available. You're good to go!")
else:
    import sys
    print("CUDAExecutionProvider is NOT available. Check your installation.")
    sys.exit()

if OPTIONS['INSTALL_COMFYUI_MANAGER']:
  %cd custom_nodes
  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

if OPTIONS['INSTALL_ANIMATEDIFF']:
  %cd ../
  ![ ! -d ComfyUI-AnimateDiff-Evolved ] && echo -= Initial setup AnimateDiff =- && git clone https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved
  %cd ComfyUI-AnimateDiff-Evolved
  !git pull

%cd $COMFY_WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !pwd
  print("-= Install custom nodes dependencies =-")
  ![ -f "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" ] && python "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py"
# might need to add other custom node dependencies here. i remember i had to manually download some


In [ ]:
#@markdown ###Download standard resources
#@markdown (NOTE: These resources are saved if "Using Google Drive" is checked, so you only need to run once).
### SDXL
### I recommend these workflow examples: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/

OPTIONS = {}

#@markdown **Models**

SDXL_1_0_BASE_AND_REFINER = False  #@param {type:"boolean"}
OPTIONS['SDXL_1_0_BASE_AND_REFINER'] = SDXL_1_0_BASE_AND_REFINER

if OPTIONS['SDXL_1_0_BASE_AND_REFINER']:
  !wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -P ./models/checkpoints/
  !wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -P ./models/checkpoints/

INFLUENCER_MODELS = False  #@param {type:"boolean"} # move this to resource models I think
OPTIONS['INFLUENCER_MODELS'] = INFLUENCER_MODELS
if OPTIONS['INFLUENCER_MODELS']:
  !wget -c https://civitai.com/api/download/models/479579 -P ./models/checkpoints/
  !wget -c https://civitai.com/api/download/models/274039 -P ./models/checkpoints/
  !wget -c https://civitai.com/api/download/models/923681 -P ./models/checkpoints/

SD_2_0_MODELS = False  #@param {type:"boolean"}
OPTIONS['SD_2_0_MODELS'] = SD_2_0_MODELS

if OPTIONS['SD_2_0_MODELS']:
  !wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors -P ./models/checkpoints/
  !wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors -P ./models/checkpoints/

SD_1_5_MODEL = False  #@param {type:"boolean"}
OPTIONS['SD_1_5_MODEL'] = SD_1_5_MODEL

if OPTIONS['SD_1_5_MODEL']:
  !wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P ./models/checkpoints/

FLUX_FP8_UNET_MODEL = False #@param {type:"boolean"}
OPTIONS['FLUX_FP8_UNET_MODEL'] = FLUX_FP8_UNET_MODEL

if OPTIONS['FLUX_FP8_UNET_MODEL']:
  !echo "Downloading flux-fp8..."
  !wget -c https://huggingface.co/Kijai/flux-fp8/resolve/main/flux-vae-bf16.safetensors -P ./models/unet/
  !wget -c https://huggingface.co/Kijai/flux-fp8/resolve/main/flux1-dev-fp8-e4m3fn.safetensors -P ./models/unet/
  !wget -c https://huggingface.co/Kijai/flux-fp8/resolve/main/flux1-dev-fp8-e5m2.safetensors -P ./models/unet/
  !wget -c https://huggingface.co/Kijai/flux-fp8/resolve/main/flux1-dev-fp8.safetensors -P ./models/unet/
  !wget -c https://huggingface.co/Kijai/flux-fp8/resolve/main/flux1-schnell-fp8-e4m3fn.safetensors -P ./models/unet/
  !wget -c https://huggingface.co/Kijai/flux-fp8/resolve/main/flux_shakker_labs_union_pro-fp8_e4m3fn.safetensors -P ./models/unet/

#@markdown **CLIP**

CLIP = False #@param {type:"boolean"}
OPTIONS['CLIP'] = CLIP

if OPTIONS['CLIP']:
  print("Downloading CLIP...")
  !wget -c https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors -P ./models/clip/
  !wget -c https://huggingface.co/mcmonkey/google_t5-v1_1-xxl_encoderonly/resolve/main/t5xxl_fp8_e4m3fn.safetensors -P ./models/clip/

#@markdown **VAEs**

SDXL_1_0_VAE = False  #@param {type:"boolean"}
OPTIONS['SDXL_1_0_VAE'] = SDXL_1_0_VAE

if OPTIONS['SDXL_1_0_VAE']:
  !wget -c https://civitai.com/api/download/models/183894 -O ./models/vae/sdxl-vae-fp16-fix.safetensors #sdxl-vae-fp16-fix.safetensors

SD_1_5_VAE = False  #@param {type:"boolean"}
OPTIONS['SD_1_5_VAE'] = SD_1_5_VAE

if OPTIONS['SD_1_5_VAE']:
  !wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/

FLUX_SCHNELL = False #@param {type:"boolean"}
OPTIONS['FLUX_SCHNELL'] = FLUX_SCHNELL

if OPTIONS['FLUX_SCHNELL']:
  print("Downloading FLUX SCHNELL...")
  !wget -c https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/ae.safetensors -P ./models/vae/

#@markdown **T2I Adapters**

T2I_ADAPTER = False  #@param {type:"boolean"}
OPTIONS['T2I_ADAPTER'] = T2I_ADAPTER

if OPTIONS['T2I_ADAPTER']:
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth -P ./models/controlnet/
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth -P ./models/controlnet/
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth -P ./models/controlnet/
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth -P ./models/controlnet/
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth -P ./models/controlnet/
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -P ./models/controlnet/
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth -P ./models/controlnet/

T2I_STYLES_MODEL = False  #@param {type:"boolean"}
OPTIONS['T2I_STYLES_MODEL'] = T2I_STYLES_MODEL

if OPTIONS['T2I_STYLES_MODEL']:
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -P ./models/style_models/

#@markdown **Controlnets**

SDXL_1_0_CONTROLNETS = False  #@param {type:"boolean"}
OPTIONS['SDXL_1_0_CONTROLNETS'] = SDXL_1_0_CONTROLNETS

if OPTIONS['SDXL_1_0_CONTROLNETS']:
  !wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-recolor-rank256.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-sketch-rank256.safetensors -P ./models/controlnet/

SD_1_5_CONTROLNETS = False  #@param {type:"boolean"}
OPTIONS['SD_1_5_CONTROLNETS'] = SD_1_5_CONTROLNETS

if OPTIONS['SD_1_5_CONTROLNETS']:
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors -P ./models/controlnet/

DEV_CONTROLNET_UNIONS = False  #@param {type:"boolean"}
OPTIONS['DEV_CONTROLNET_UNIONS'] = DEV_CONTROLNET_UNIONS

if OPTIONS['DEV_CONTROLNET_UNIONS']:
  # %mkdir flux # the german guy said to put it in a new flux folder. i tried ignoring it!
  # First beta version checkpoint. Need to update in future. For now don't need?
  # !wget -c https://huggingface.co/InstantX/FLUX.1-dev-Controlnet-Union/resolve/main/diffusion_pytorch_model.safetensors -P ./models/controlnet/

  !wget -c https://huggingface.co/Shakker-Labs/FLUX.1-dev-ControlNet-Union-Pro/resolve/main/diffusion_pytorch_model.safetensors -P ./models/controlnet/

#@markdown **Upscalers**

ESRGAN_UPSCALER = False  #@param {type:"boolean"}
OPTIONS['ESRGAN_UPSCALER'] = ESRGAN_UPSCALER

if OPTIONS['ESRGAN_UPSCALER']:
  !wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P ./models/upscale_models/
  !wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P ./models/upscale_models/
  !wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P ./models/upscale_models/

CLEAR_REALITY_V1 = False  #@param {type:"boolean"}
OPTIONS['CLEAR_REALITY_V1'] = CLEAR_REALITY_V1

if OPTIONS['CLEAR_REALITY_V1']:
  print("ATTENTION!!!")
  print("===================================")
  print("ENSURE YOU HAVE CLEAR_REALITY_V1 DOWNLOADED")
  print("Link: https://openmodeldb.info/models/4x-ClearRealityV1")
  print("Put it inside models/upscale_models")
  print("===================================")

  # !wget -c -P ./models/upscale_models/

#@markdown **ContrastiveAlignment**
PURE_AND_LIGHTING_ID_CUSTOMIZATION = False  #@param {type:"boolean"}
OPTIONS['PURE_AND_LIGHTING_ID_CUSTOMIZATION'] = PURE_AND_LIGHTING_ID_CUSTOMIZATION

if OPTIONS['PURE_AND_LIGHTING_ID_CUSTOMIZATION']:
  print("Downloading Pure and Lighting ID Customization...")
  !wget -c https://huggingface.co/guozinan/PuLID/resolve/main/pulid_flux_v0.9.0.safetensors -P ./models/pulid/

#@markdown **LORAs
FLUX_TURBO_ALPHA = False  #@param {type:"boolean"}
OPTIONS['FLUX_TURBO_ALPHA'] = FLUX_TURBO_ALPHA
if OPTIONS['FLUX_TURBO_ALPHA']: # might need to go into a new "lora" folder
  print("Downloading 8-step-flux.safetensors...")
  !wget -c https://huggingface.co/alimama-creative/FLUX.1-Turbo-Alpha/resolve/main/diffusion_pytorch_model.safetensors -P ./models/loras/

#@markdown **AnimateDiff**

AD_MOTION_MODELS = False  #@param {type:"boolean"}
OPTIONS['AD_MOTION_MODELS'] = AD_MOTION_MODELS

if OPTIONS['AD_MOTION_MODELS']:
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v14.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/models/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/models/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/models/

AD_MOTION_LORAS = False  #@param {type:"boolean"}
OPTIONS['AD_MOTION_LORAS'] = AD_MOTION_LORAS

if OPTIONS['AD_MOTION_LORAS']:
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_PanLeft.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_PanRight.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_RollingAnticlockwise.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_RollingClockwise.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_TiltDown.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_TiltUp.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomIn.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomOut.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/

# need to figure out how to classify these
OTHER = True  #@param {type:"boolean"}
OPTIONS['OTHER'] = OTHER
if OPTIONS['OTHER']:
  # might be wrong filepath/double nested
  !wget -c https://huggingface.co/MonsterMMORPG/tools/resolve/main/antelopev2.zip -P ./models/insightface/models/ && unzip -o ./models/insightface/models/antelopev2.zip -d ./models/insightface/models/antelopev2/ && rm ./models/insightface/models/antelopev2.zip
